In [1]:
from utils.lexical import normalizador
normalizer = normalizador.Normalizador()
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import os
import re

/home/marcio/.virtualenvs/AvaliacoesGCC151_env/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/home/marcio/.virtualenvs/AvaliacoesGCC151_env/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


# Question 3 - Getting corpora data and normalizing

## Getting corpora files

In [2]:
corpora_path = '../data/corpora/'
corpora_dirs = os.listdir(corpora_path)

corpora_files = {} #elements are like corpus:[files]
for corpus in corpora_dirs:
    corpora_files[corpus] = [os.path.join(corpora_path + corpus, f)
             for f in os.listdir(corpora_path + corpus)
             if os.path.isfile(os.path.join(corpora_path + corpus, f))]

## Getting data to construct Word2Vec and Doc2Vec

Here we are going to collect the data in the following ways:
* A list of all sentences(tokenized by words) for each corpus
* A list of documents(tokenized by words) for the entire corpora

The data will be normalized by transforming the text to lowercase and removing stopwords.

In [3]:
corpora_sentences = {} #elements are like: corpus:[sentences]
corpora_documents = [] #elements are like: documents tokenized by words
for corpus in corpora_files:
    corpora_sentences[corpus] = []
    for file in corpora_files[corpus]:
        with open(file, 'r') as text_file:
            #all files in my corpora have only one line
            #because I removed the line brakes during the compilation
            text = text_file.read()
            
            #removing trash read from the sports blog
            if corpus == "esporte":
                text = re.sub("^Pesquisar este blog ", '', text)
            
            #transforming to lowercase
            text = normalizer.to_lowercase(text)
            
            #tokenizing by word and removing stopwords - this is for Doc2Vec
            document = normalizer.tokenize_words(text)
            document = normalizer.remove_stopwords(document)
            corpora_documents.append(document)
            
            #tokenizes text by sentence - this is for Word2Vec
            sentences = normalizer.tokenize_sentences(text)
            #tokenizes each sentence by word, removes stopwords and adds it to the list
            sentences = [normalizer.remove_stopwords(normalizer.tokenize_words(sent)) for sent in sentences]
            corpora_sentences[corpus].extend(sentences)
    
    #this is also for Doc2Vec
    tagged_documents = [TaggedDocument(words=d, tags=[str(i)]) 
                        for i, d in enumerate(corpora_documents)]

# 3.a Creating Word2Vecs for each corpus

### Training the Word2Vecs

In [4]:
w2vmodels = {} #elements are like corpus: w2vmodel
for corpus in corpora_sentences:
    w2vmodels[corpus] = Word2Vec(corpora_sentences[corpus],
        size=200, window=5, min_count=3, workers=4)
        

# 3.b Creating Doc2Vec for the corpora

In [5]:
d2vmodel = Doc2Vec(tagged_documents, vector_size=20, window=2, min_count=1, workers=4)

# Question 4

## 4.a.i.1 - Using Word2Vec

### Ex. 1:

In [6]:
w2vmodels['esporte'].wv.most_similar('jogo', topn=10)

[('fez', 0.9999263286590576),
 ('palmeiras', 0.999906063079834),
 ('ficou', 0.9999042749404907),
 ('contra', 0.9999023675918579),
 ('fluminense', 0.9999018907546997),
 ('placar', 0.9999004006385803),
 ('perdeu', 0.9998984336853027),
 ('duelo', 0.9998971223831177),
 ('méxico', 0.9998952746391296),
 ('fim', 0.9998934864997864)]

### Ex. 2:

In [7]:
w2vmodels['ciencia_e_tecnologia'].wv.most_similar('energia', topn=10)

[('alternada', 0.9705200791358948),
 ('eletricidade', 0.9652921557426453),
 ('corrente', 0.96380215883255),
 ('neve', 0.963367760181427),
 ('fervente', 0.9607200622558594),
 ('silicone', 0.9594526290893555),
 ('produzindo', 0.9569694995880127),
 ('eletromagnética', 0.956891655921936),
 ('entra', 0.9567195177078247),
 ('dispositivo', 0.9546564817428589)]

### Ex. 3:

In [8]:
w2vmodels['ciencia_e_tecnologia'].wv.most_similar('átomo', topn=10)

[('hidrogênio', 0.9975570440292358),
 ('carregado', 0.9948441982269287),
 ('emitido', 0.9931716918945312),
 ('orbitar', 0.991255521774292),
 ('recombinam', 0.9912310242652893),
 ('saída', 0.9910465478897095),
 ('metálica', 0.9888768196105957),
 ('atômico', 0.9887674450874329),
 ('pulso', 0.9881608486175537),
 ('orbita', 0.9880216121673584)]

Do ponto de vista morfológico, algumas medidas podem ser tomadas para tentar aprimorar os restultados da aplicação do Word2Vec. O que foi empregado acima foi a remoção de palavras não-significativas, já que essas palavras cumprem apenas uma função estrutual no texto e pouco contribuem para o significado. Uma outra medida que possivelmente melhoraria os resultados obtidos seria lematizar as palavras antes de iniciar a análise, pois, assim, duas ocorrências com diferentes "formas" da mesma palavra contariam como o mesmo contexto. Essas são questões abertas à pesquisa e ao debate, e diferentes implementações de procedimentos para criar vetores de palavras baseadas em contagem tomam decisões diferentes sobre essa questão.

## 4.a.i.2 - Using Word2Vec

O Word2Vec é normalmente utilizado para comparar o significado de palavaras. Com o intuito de comparar documentos inteiros, a abordagem pensada consiste em obter o vetor médio de todas as palavras de cada documento e utilizar esse valor como a representação do "significado" de cada documento.

* Creating the document vectors:

In [9]:
import numpy as np

w2v_sci = w2vmodels['ciencia_e_tecnologia']
w2v_spo = w2vmodels['esporte']

sci1 = [word for word in corpora_documents[0] if word in w2v_sci.wv.vocab]
vec_sci1 = np.mean(w2v_sci.wv.__getitem__(sci1), axis=0)

sci2 = [word for word in corpora_documents[1] if word in w2v_sci.wv.vocab]
vec_sci2 = np.mean(w2v_sci.wv.__getitem__(sci2), axis=0)

spo1 = [word for word in corpora_documents[-1] if word in w2v_spo.wv.vocab]
vec_spo1 = np.mean(w2v_spo.wv.__getitem__(spo1), axis=0)

spo2 = [word for word in corpora_documents[-2] if word in w2v_spo.wv.vocab]
vec_spo2 = np.mean(w2v_spo.wv.__getitem__(spo2), axis=0)


* Creating a function to calculate the cosine-similarity:

In [10]:
import math

def cosine_similarity(vec1, vec2):
    if len(vec1) != len(vec2):
        return "Error: the vectors must have the same number of dimensions"

    dot_product = 0
    norm1, norm2 = 0, 0
    for i in range(len(vec1)):
        dot_product += vec1[i] * vec2[i]
        norm1 += vec1[i]**2
        norm2 += vec2[i]**2
    
    norm1 = math.sqrt(norm1)
    norm2 = math.sqrt(norm2)
    
    cos_sim = dot_product / (norm1 * norm2)
    return cos_sim
        

* Calculating the cosine-similarity:

In [11]:
print(cosine_similarity(vec_sci1, vec_sci2))
print(cosine_similarity(vec_spo1, vec_spo2))
print(cosine_similarity(vec_sci2, vec_spo1))

#testing the function
v1 = [1, 1]
v2 = [-1, -1]

print('\n')
print(cosine_similarity(v1, v2))
print(cosine_similarity(v1, v1))

0.9306020439938492
0.9999870967358836
0.3286927347546636


-0.9999999999999998
0.9999999999999998


Os resultados mostram diferenças apenas a partir da terceira casa decimal; desconsiderando as magnitudes dessas diferenças, a ocorrência delas parece fazer um certo sentido, suspeito que a forma de calcular um vetor para o documento deva ser aprimorada para produzir diferenças mais significativas.

## 4.b - Using Doc2Vec

* Creating a function to find the most similar document in a specific corpus:

In [12]:
def most_similar_doc(d2v, corpus_docs, model_d2v):
    from scipy import spatial
    
    smallest_dist = 1
    most_similar = None
    
    for doc in corpus_docs:
        doc2v = d2vmodel.infer_vector(doc)
        cos_dist = spatial.distance.cosine(d2v, doc2v)
        if(cos_dist < smallest_dist):
            most_similar = doc
            smallest_dist = cos_dist
    
    return most_similar

In [13]:
sci_docs = corpora_documents[:500]
spo_docs = corpora_documents[500:]

d2v_sci1 = d2vmodel.infer_vector(sci_docs[0])
d2v_sci2 = d2vmodel.infer_vector(sci_docs[1])
d2v_spo1 = d2vmodel.infer_vector(spo_docs[499])

most_similar = most_similar_doc(d2v_sci1, spo_docs, d2vmodel)

print("Sci doc:\n\n{}\n".format(sci_docs[0]))
print("Most similar spo doc found:\n\n{}".format(most_similar))

Sci doc:

['físicos', 'detectaram', 'assinatura', 'reações', 'fusão', 'nuclear', 'aparato', 'experimental', 'mesa', ',', 'comumente', 'usado', 'estudar', 'plasmas', 'encontrados', 'estrelas', 'outros', 'corpos', 'astrofísicos', '.', 'futuros', 'reatores', 'fusão', 'nuclear', 'prometem', 'possibilidade', 'fornecer', 'terra', 'fonte', 'ilimitada', 'energia', 'limpa', ',', 'verdadeiras', 'estrelas', 'artificiais', '.', 'tentativas', 'criar', 'reatores', 'fusão', 'nuclear', 'normalmente', 'envolvem', 'engenhocas', 'tamanho', 'edifícios', 'gerar', 'plasma', 'quente', 'necessário', 'iniciar', 'reações', 'fusão', '-', 'é', 'caso', 'projetos', 'experimentais', 'iter', 'wendelstein', '7x', 'propostas', 'ambiciosas', 'curto', 'prazo', ',', 'sparc', 'hb11', '.', 'yue', 'zhang', ',', 'universidade', 'washington', ',', 'eua', ',', 'acredita', 'ter', 'iniciado', 'sucesso', 'fusão', 'nuclear', 'sustentada', 'usando', 'configuração', 'é', 'pequena', 'suficiente', 'ser', 'posta', 'sobre', 'mesa', '.', 

In [14]:
most_similar = most_similar_doc(d2v_spo1, sci_docs, d2vmodel)

print("Spo doc:\n\n{}\n".format(spo_docs[499]))
print("Most similar sci doc found:\n\n{}".format(most_similar))

Spo doc:

['cob', 'divulga', 'candidatos', 'esportistas', 'ano', ';', 'veja', 'nadador', 'cesar', 'cielo', ',', 'judoca', 'leandro', 'guilheiros', 'ponteiro', 'murilo', 'enfres', ',', 'seleção', 'brasileira', 'masculina', 'vôlei', ',', 'três', 'candidatos', 'troféu', 'melhor', 'esportista', 'ano', 'masculino', '.', 'feminino', 'disputa', 'ana', 'marcela', 'cunha', '(', 'maratonas', 'aquáticas', ')', ',', 'fabiana', 'murer', '(', 'atletismo', ')', 'dupla', 'juliana', 'larissa', '(', 'vôlei', 'praia', ')', '.', 'concorrentes', 'anunciados', 'comitê', 'olímpico', 'brasileiro', '(', 'cob', ')', 'nesta', 'sexta-feira', '.', 'torcedores', 'poderão', 'votar', 'partir', 'próxima', 'terça-feira', 'site', 'oficial', 'cob', '(', 'www.cob.com.br', ')', 'escolherem', 'campeão', 'duas', 'eleições', '.', 'anúncio', 'vencedores', 'acontecerá', 'dia', '20', 'dezembro', ',', 'durante', 'cerimônia', 'prêmio', 'brasil', 'olímpico', ',', 'teatro', 'mam', ',', 'rio', 'janeiro', '.', 'júri', 'composto', 'jor

In [15]:
most_similar = most_similar_doc(d2v_sci2, spo_docs, d2vmodel)

print("Sci doc:\n\n{}\n".format(sci_docs[1]))
print("Most similar spo doc found:\n\n{}".format(most_similar))

Sci doc:

['vasile', 'palade', '(', 'universidade', 'coventry', ')', 'gerry', 'wolff', '(', 'cognition', 'research', ')', ',', 'acreditam', 'é', 'possível', 'começar', 'desenvolver', 'novo', 'modelo', 'inteligência', 'artificial', 'supere', 'deficiências', 'sistemas', 'atuais', '.', 'chamam', 'atenção', 'limitações', 'aprendizagem', 'profunda', ',', '``', 'esquecimento', 'catastrófico', "''", ',', 'significa', ',', 'sistema', 'aprendizagem', 'profunda', 'aprende', 'algo', 'novo', ',', 'novo', 'aprendizado', 'pode', 'eliminar', 'tudo', 'sistema', 'aprendeu', 'antes', '.', 'levantado', 'preocupação', 'crescente', 'sobre', 'aprendizagem', 'profunda', 'caminho', 'desenvolvimento', 'inteligência', 'artificial', 'consiga', 'competir', 'inteligência', 'humana', '.', 'dupla', 'propõe', 'outra', 'rota', ':', 'teoria', 'inteligência', 'sp', ',', 'desenvolvida', 'combinar', 'simplicidade', '(', 's', ')', 'estrutura', 'funcionamento', 'poder', '(', 'p', ')', 'descritivo', 'explicativo', 'gama', 'r

Os resultados apresentados são difíceis de serem comparados, uma vez que os assuntos trados por textos dos diferents corpora são bastante distintos, mas podem ser observadas algumas palavras similares e algumas expressões com sentidos parecidos. Como o Doc2Vec também constói vetores baseado no contexto das palavras, talvez seja possível, assim como no caso do Word2Vec usdo para comparar documentos, que a lematização dos textos antes da execução da comparação possa melhorar os resultados, fazendo com que diferentes formas da mesma palavra passem a representar o mesmo contexto.